In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
from scipy.stats import ttest_ind
import numpy as np
from scipy.stats.stats import pearsonr
from IPython.display import display, Markdown, Latex
plt.style.use('ggplot')
from datetime import date
data_path = '/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/'
import dask.dataframe as dd
from scipy.stats import norm
from sklearn.cluster import KMeans
import numpy as np

import pandas



/var/folders/8g/qp1k_1g91m5f2m9pypxwxdhh0000gn/T/ipykernel_56982/3471708860.py:8: DeprecationWarning: Please use `pearsonr` from the `scipy.stats` namespace, the `scipy.stats.stats` namespace is deprecated.
  from scipy.stats.stats import pearsonr


In [2]:
#load mobility and temperature data
#load heatwave - control dataframe
mobility_heat = pd.read_csv('/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/paper2/turkey/THI_mobility_heat_clean_90.csv')
heatwave_control = pd.read_csv('/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/paper2/turkey/THI_heatwave_control_90.csv')

## Build Intensity Variable


In [3]:
full_df = pd.read_csv('/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/daily_max/Turkey/THI_daily_max_turkey.csv')

full_df.rename(columns={'time': 'datetime'}, inplace=True)
full_df['datetime'] = pd.to_datetime(full_df['datetime'])
full_df['day'] = pd.DatetimeIndex(full_df['datetime']).day
full_df['month'] = pd.DatetimeIndex(full_df['datetime']).month
full_df['year'] = pd.DatetimeIndex(full_df['datetime']).year
full_df = full_df[~full_df['month'].isin([4, 10])]
full_df['location'] = full_df['adm2'] + ', ' + full_df['adm1']
full_df['datetime_ord'] = [date(x,y,z).toordinal() for x,y,z in zip(full_df['year'], full_df['month'], full_df['day'])]

In [4]:
mobility_heat['control_day'] = pd.to_datetime(mobility_heat['control_day'])

In [5]:
#merge temperatures from control day onto mobility heat
mobility_heat = mobility_heat.merge(full_df[['datetime', 'thi', 'location']], left_on=['location', 'control_day'], right_on=['location', 'datetime'], suffixes=('_heat', '_control'))

In [6]:
(mobility_heat['thi_heat'] - mobility_heat['thi_control']).max()

13.391376071506073

In [7]:
np.corrcoef(mobility_heat['thi_heat'], mobility_heat['thi_control'])


array([[1.        , 0.90839647],
       [0.90839647, 1.        ]])

### Create/explore temperature difference variable

In [8]:
mobility_heat['temp_diff'] = mobility_heat['thi_heat'] - mobility_heat['thi_control']

In [9]:
mobility_heat['temp_diff'].describe()

count    6298.000000
mean        4.000642
std         1.629398
min         0.921252
25%         2.782182
50%         3.782660
75%         4.996801
max        13.391376
Name: temp_diff, dtype: float64

In [10]:
sns.histplot(mobility_heat, x='temp_diff')
plt.title("Temperature Difference (heat wave vs. control)")
plt.tight_layout()
plt.savefig(data_path + '/img/12_09_2023/india_temp_diff_dist.png')

### Build Duration Variable
* Duration of heat wave (in days)
* Build variable showing which day of a heatwave it is

In [11]:
# Formatting column names and merging dataframes

In [12]:
heatwave_control.sort_values(by='Heat Wave Duration', inplace=True)

In [13]:
duration_IDs = heatwave_control.drop_duplicates('heat_ID')[['heat_ID', 'Heat Wave Duration']  ]             

In [14]:
duration_IDs.columns = ['heat_ID', 'Heat Wave Duration']

In [15]:
duration_IDs.head()

,heat_ID,Heat Wave Duration
0,77ee484e-d00d-450d-843e-d46ba3739cdb,2.0
9860,f393b340-39bd-4c6c-8e6d-ae987f8338dd,2.0
3903,b253edcd-3337-4cd1-8176-79aa5ac3b5cd,2.0
9886,24810d5c-a257-4b19-820e-58ca4b3a89d3,2.0
3888,6ddc159a-9fbb-4c94-b65a-b7b77b843a50,2.0


In [16]:
duration_IDs['Heat Wave Duration'].describe()

count    3531.000000
mean        4.208156
std         2.571582
min         2.000000
25%         2.000000
50%         3.000000
75%         5.000000
max        23.000000
Name: Heat Wave Duration, dtype: float64

In [17]:
#drop original heatwave duration variable
heatwave_control.drop('Heat Wave Duration', axis=1, inplace=True)

In [18]:
#merge duration ID dataframe onto heatwave control
heatwave_control = heatwave_control.merge(duration_IDs, on='heat_ID', how='left')

In [19]:
heatwave_control.drop_duplicates(['location', 'datetime'], inplace=True)

In [20]:
heatwave_control['dur_ranking'] = heatwave_control['Heat Wave Duration'].rank(pct=True)

In [21]:
#groupby each heatwave, sort by date, assign increasing rank for each passing day
heat_stage = heatwave_control.groupby('heat_ID').apply(lambda x: x.datetime.sort_values().rank()).reset_index()

In [22]:
heat_stage.columns=['heat_ID', 'index', 'stage']

In [23]:
heat_stage['stage'].max()

23.0

In [24]:
x = heat_stage.merge(duration_IDs, on='heat_ID')

In [25]:
heatwave_control.drop('index', axis=1, inplace=True)

In [26]:
heatwave_control.reset_index(inplace=True)

In [27]:
heatwave_control = heatwave_control.merge(heat_stage, on=['index', 'heat_ID'])

In [28]:
heatwave_control['datetime'] = pd.to_datetime(heatwave_control['datetime'])

In [29]:
heatwave_control['stage'].max()

23.0

In [30]:
mobility_heat['date_heat'] = pd.to_datetime(mobility_heat['date_heat'])

In [31]:
#merge temporal variables into mobility_heat

mobility_heat = mobility_heat.merge(heatwave_control[['location', 'datetime', 'Heat Wave Duration', 'dur_ranking', 'stage']], left_on=['location', 'date_heat'], right_on=['location', 'datetime'])

In [32]:
mobility_heat.reset_index(inplace=True, drop=True)

In [33]:
mobility_heat['month'] = [x.month for x in mobility_heat['datetime']]

In [34]:
mobility_heat['month'].unique()

array([8, 7, 9, 6, 5])

#### Flexible early heatwave definition based on percentile

Gets first X percentile of country-wide heatwaves

In [39]:
# format final variables
mobility_heat['stage_frac'] = (mobility_heat['stage'] / mobility_heat['Heat Wave Duration_y'] )
mobility_heat.drop_duplicates('heat_ID')['Heat Wave Duration_y'].describe()


count    1660.000000
mean        3.978916
std         2.436692
min         2.000000
25%         2.000000
50%         3.000000
75%         5.000000
max        19.000000
Name: Heat Wave Duration_y, dtype: float64

In [40]:
mobility_heat['heatwave_dur'] = mobility_heat['Heat Wave Duration_y'] >= 5

In [41]:
mobility_heat['month'] = [x.month for x in pd.to_datetime(mobility_heat['date_control'])]

In [42]:
mobility_heat['date_control'] = pd.to_datetime(mobility_heat['date_control'])

In [43]:
mobility_heat['date_control']

0      2022-08-02
1      2022-08-03
2      2022-08-02
3      2022-08-03
4      2022-08-15
          ...    
6293   2022-08-16
6294   2022-07-25
6295   2022-07-27
6296   2022-08-16
6297   2022-08-25
Name: date_control, Length: 6298, dtype: datetime64[ns]

## Merge social and physical variables

In [44]:
soc_phys = gpd.read_file('/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/physical/turkey/merged.shp')

In [45]:
mobility_heat= mobility_heat.merge(soc_phys[['location', 'mean_elev', 'coast', 'HDI_dec', 'popden_dec', 'over60_dec']], on='location')

In [46]:
mobility_heat['elev_1500m'] = mobility_heat['mean_elev'] >= 1500

In [47]:
mobility_heat['elev_1500m'] = mobility_heat['elev_1500m'] * 1

### Remove holidays 

In [48]:
import requests

In [49]:
hols_2022 = pd.DataFrame(requests.get("https://date.nager.at/api/v3/publicholidays/2022/TR").json())
hols_2022['date'] = pd.to_datetime(hols_2022['date'])

In [50]:
mobility_heat = mobility_heat[~mobility_heat['datetime_heat'].isin(hols_2022['date'].values)]
mobility_heat = mobility_heat[~mobility_heat['datetime_control'].isin(hols_2022['date'].values)]

## Normalized rain variable for heatwave and control day

In [57]:
precip = pd.read_csv('/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/precipitation/turkey/daily_sum_precipitation.csv')
precip.rename(columns={'valid_time': 'time'}, inplace=True)
precip['time'] = pd.to_datetime(precip['time'])

In [59]:
gdf = gpd.read_file('/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/adm0_shp/turkey/clean_shp.shp')
gdf.to_crs(epsg='3857', inplace=True)
gdf['area'] = [x.area for x in gdf['geometry']]

In [60]:
#area from m --> km
#rain from m --> mm
precip['daily_precipitation'] = precip['daily_precipitation'] * 1000
gdf['area'] = gdf['area'] / 1000

In [61]:
precip = precip.merge(gdf[['location', 'area']], on='location')
precip['rain_normed'] = (precip['daily_precipitation'] / precip['area']) 

In [62]:
precip['rain_standard'] = precip.groupby('location')['rain_normed'].rank(pct=True) * 100

### Control day rain dummy

In [63]:
mobility_heat = mobility_heat.merge(precip[['time', 'adm1', 'adm2', 'rain_standard']], left_on=['control_day', 'adm1', 'adm2'], right_on=['time', 'adm1', 'adm2'])

In [64]:
mobility_heat.rename(columns={'rain_standard': "rain_standard_control"}, inplace=True)

### Heatwave day rain dummy

In [65]:
mobility_heat = mobility_heat.merge(precip[['time', 'adm1', 'adm2', 'rain_standard']], left_on=['date_heat', 'adm1', 'adm2'], right_on=['time', 'adm1', 'adm2'])

In [66]:
mobility_heat.rename(columns={'rain_standard': "rain_standard_heat"}, inplace=True)

### Prepare final dataset

In [67]:
df = mobility_heat.copy()

In [68]:
df = df[df['mean_elev'] <= 2500]

In [69]:
#set outlier cutoff (how many SDs away from mean)
SD = 4

In [70]:
for variable in ['workplaces_difference', 'residential_difference', 'grocery_pharm_difference', 'parks_difference', 'retail_rec_difference', 'transit_difference']:
    df[variable] = [np.nan if (x >= df[variable].mean() + (SD * df[variable].std())) | (x <= df[variable].mean() - (SD * df[variable].std())) else x for x in df[variable] ]

    #v13
df.dropna(how='all', subset=['workplaces_difference', 'residential_difference', 'transit_difference', 'grocery_pharm_difference', 'parks_difference', 'retail_rec_difference'], inplace=True)

In [71]:
df = df.iloc[:, 3:]

In [72]:
df.reset_index(inplace=True)

In [73]:
df[['residential_difference', 'workplaces_difference', 'transit_difference', 'grocery_pharm_difference', 'retail_rec_difference', 'parks_difference']].describe().transpose()[['count', 'mean', 'std', 'min', 'max']]

,count,mean,std,min,max
residential_difference,3598.0,0.595609,2.622194,-10.0,11.0
workplaces_difference,5970.0,1.177554,11.561397,-54.0,59.0
transit_difference,2519.0,-1.633982,15.261482,-75.0,66.0
grocery_pharm_difference,3007.0,0.793482,12.733890,-60.0,62.0
retail_rec_difference,3041.0,1.178231,9.818809,-43.0,48.0
parks_difference,2429.0,-0.366406,21.714524,-110.0,97.0


In [74]:
df['datetime_heat'].min()

'2022-05-29'

In [76]:
df.to_csv(f'/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/paper2/turkey/THI_mobility_heat_w_vars_90_{SD}SD_precip.csv')